## Ola-7b

* Doesn't output bounding boxes when asked

In [1]:
import os

os.environ['LOWRES_RESIZE'] = '384x32'
os.environ['HIGHRES_BASE'] = '0x32'
os.environ['VIDEO_RESIZE'] = "0x64"
os.environ['VIDEO_MAXRES'] = "480"
os.environ['VIDEO_MINRES'] = "288"
os.environ['MAXRES'] = '1536'
os.environ['MINRES'] = '0'
os.environ['FORCE_NO_DOWNSAMPLE'] = '1'
os.environ['LOAD_VISION_EARLY'] = '1'
os.environ['PAD2STRIDE'] = '1'

import sys
# Add the desired directory to the Python path
sys.path.append(os.path.abspath('/data/students/earl/llava-dissector/Ola'))

import torch
import re
from PIL import Image
import numpy as np
import transformers
from ola.conversation import conv_templates, SeparatorStyle
from ola.model.builder import load_pretrained_model
from ola.datasets.preprocess import tokenizer_image_token
from ola.mm_utils import KeywordsStoppingCriteria, process_anyres_highres_image
from ola.constants import IGNORE_INDEX, DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
import requests
import io


'''
parser = argparse.ArgumentParser()
parser.add_argument('--model_path', type=str, default='THUdyh/Ola-7b')
parser.add_argument('--text', type=str, default=None)
parser.add_argument('--image_path', type=str, default=None)
args = parser.parse_args()
'''

model_path = '/data/students/earl/llava-dissector/Ola/Ola-7b'
tokenizer, model, image_processor, _ = load_pretrained_model(model_path, None)
#model = model.to('cuda').eval()
#model = model.bfloat16()
model.eval()

#cur_dir = os.path.dirname(os.path.abspath(__file__))

'''
image_path = #args.image_path
text = #args.text

if image_path is not None:
    visual = image_path
    modality = "image"
else:
    modality = "text"

if modality == "image":
    image = [Image.open(visual)]
    image_sizes = [image[0].size]
else:
    images = [torch.zeros(1, 3, 224, 224).to(dtype=torch.bfloat16, device='cuda', non_blocking=True)]
    images_highres = [torch.zeros(1, 3, 224, 224).to(dtype=torch.bfloat16, device='cuda', non_blocking=True)]
    image_sizes = [(224, 224)]
'''
def load_image(image_file):
    # Check if image_file is a URL
    if isinstance(image_file, str) and (image_file.startswith('http://') or image_file.startswith('https://')):
        response = requests.get(image_file)
        image = Image.open(io.BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

image_path = "https://farm3.staticflickr.com/2402/2480652763_e6b62303ee_z.jpg"
image = load_image(image_path)

cls = "baseball bat"
#text = f'Give the bounding box coordinates of all instances of {cls} in the image.'
text = f'Locate the {cls} in the image.'
conv_mode = "qwen_1_5"
if image_path: # image
    qs = DEFAULT_IMAGE_TOKEN + "\n" + text
elif text: # text
    qs = text

conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
device = next(model.parameters()).device

if image_path:
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
else:
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)

# Process the image
images = [image]
image_sizes = [image.size]

image_processor.do_resize = False
image_processor.do_center_crop = False
image_tensor, image_highres_tensor = [], []
for visual in images:
    image_tensor_, image_highres_tensor_ = process_anyres_highres_image(visual, image_processor)
    image_tensor.append(image_tensor_)
    image_highres_tensor.append(image_highres_tensor_)
if all(x.shape == image_tensor[0].shape for x in image_tensor):
    image_tensor = torch.stack(image_tensor, dim=0)
if all(x.shape == image_highres_tensor[0].shape for x in image_highres_tensor):
    image_highres_tensor = torch.stack(image_highres_tensor, dim=0)
if type(image_tensor) is list:
    image_tensor = [_image.bfloat16() for _image in image_tensor]
else:
    image_tensor = image_tensor.bfloat16()
if type(image_highres_tensor) is list:
    image_highres_tensor = [_image.bfloat16() for _image in image_highres_tensor]
else:
    image_highres_tensor = image_highres_tensor.bfloat16()

pad_token_ids = 151643

attention_masks = input_ids.ne(pad_token_ids).long()
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

gen_kwargs = {}

if "max_new_tokens" not in gen_kwargs:
    gen_kwargs["max_new_tokens"] = 1024
if "temperature" not in gen_kwargs:
    gen_kwargs["temperature"] = 0.2
if "top_p" not in gen_kwargs:
    gen_kwargs["top_p"] = None
if "num_beams" not in gen_kwargs:
    gen_kwargs["num_beams"] = 1

print("input_ids shape:", input_ids.shape)
print("input_ids max:", input_ids.max().item())
print("input_ids min:", input_ids.min().item())
print("vocab size:", model.vocab_size)
assert input_ids.max().item() < model.vocab_size, "input_ids out of bounds!"
print("input_ids device:", input_ids.device)
print("model device:", next(model.parameters()).device)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids=input_ids,
        images=image_tensor,
        images_highres=image_highres_tensor,
        image_sizes=image_sizes,
        modalities=['image'],
        attention_mask=attention_masks,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
        do_sample=True if gen_kwargs["temperature"] > 0 else False,
        temperature=gen_kwargs["temperature"],
        top_p=gen_kwargs["top_p"],
        num_beams=gen_kwargs["num_beams"],
        max_new_tokens=gen_kwargs["max_new_tokens"],
    )

outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
outputs = outputs.strip()
if outputs.endswith(stop_str):
    outputs = outputs[:-len(stop_str)]
outputs = outputs.strip()

print(outputs)

/data/students/earl/anaconda3/envs/rca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-11 18:35:11,513] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/data/students/earl/anaconda3/envs/rca/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/data/students/earl

[2025-06-11 18:35:12,570] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
LOAD_VISION_EARLY is set
FORCE_NO_DOWNSAMPLE is set
VIDEO_RESIZE is set as 0x64, 0, 64
HIGHRES_BASE is set as 0x32, 0, 32
MAXRES is set as 1536
MINRES is set as 0
VIDEO_MAXRES is set as 480
VIDEO_MINRES is set as 288
PAD2STRIDE is set
LOWRES_RESIZE is set as 384x32
Buiding OryxViTWrapper from ./pretrained/oryx_vit.pth...
Loading vision model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.10it/s]


Loading vision tower...
Loading vision tower succeeded.
input_ids shape: torch.Size([1, 30])
input_ids max: 151645
input_ids min: 13
vocab size: 151670
input_ids device: cuda:0
model device: cuda:0
system
You are a helpful assistant.
user
⽗
Locate the baseball bat in the image.
assistant
The baseball bat is located on the left side of the image, held by the person wearing a white shirt and black pants.
